In [1]:
import pandas as pd
import numpy as np
from scipy import stats
from collections import Counter
from evaluate import f1score
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import RandomForestClassifier
import pylab
pylab.rcParams['figure.figsize'] = (10, 6)

In [3]:
#
# Read in training (and validation) data
# 

print "Reading and merging orders tables ... ",

train = pd.read_csv('order_products__train.csv')
prior = pd.read_csv('order_products__prior.csv')
orders = pd.read_csv('orders.csv')
products = pd.read_csv('products.csv')

train = train.merge(
    orders, on='order_id', how='left').merge(
    products, on='product_id', how='left')
prior = prior.merge(
    orders, on='order_id', how='left').merge(
    products, on='product_id', how='left')

# compute training target
target = train.groupby('user_id'
    ).apply(lambda x: set(x[x['reordered'] == True]['product_id'])
    ).reset_index(name='target')

print "done."

 Reading and merging orders tables ...  done.


In [31]:
#
# Before splitting off the validation set, compute "product features".
# These are features that depend only on product_id; their values are the
# same for training, validation and testing.
#
nprods = len(products['product_id'].unique())
prod_features = pd.DataFrame(
    index=range(nprods),
    columns=['product_id'])
prod_features['product_id'] = products['product_id'].unique()
prod_features.head()

,product_id
0,1
1,2
2,3
3,4
4,5


In [32]:
#
#   2: Department reorder probability
#
x = prior.groupby('department_id').apply(
    lambda x: sum(x['reordered']) / 1.0 / len(x) ).reset_index(name='dept_reorder_prob')
x = x.merge(products, on='department_id', how='left')

prod_features = prod_features.merge(
    x[['product_id', 'dept_reorder_prob']], on='product_id', how='left')

prod_features.head()
#
# CHECKME: Why do these numbers seem so different from (1)??
#

#
# ANSWER: Because you are computing slightly different things. The
# query below gives an number more similar to the category reorder
# rate. See also below where I now compute prod prob differently.
#

#   number of users who ever had this product in their basket
# x = prior.groupby('product_id').apply(
#    lambda x: sum(x['reordered']) / 1.0 / len(x)).reset_index(name='reorder_prob2')

# prod_features = prod_features.merge(
#    x[['product_id', 'reorder_prob2']], on='product_id', how='left')
# prod_features.head()

,product_id,dept_reorder_prob
0,1,0.574180
1,2,0.346721
2,3,0.653460
3,4,0.541885
4,5,0.346721


In [33]:
#
#   3: Department ID heuristic split
#
prod_features = prod_features.merge(
    products[['product_id', 'department_id']], on='product_id', how='left')
prod_features['department_id'] = prod_features.apply(
    lambda x: 1 if x['department_id'] in [4, 16] else 0, axis=1)
prod_features.head()

# sanity check
# prod_features[prod_features['department_id'] == 1].merge(
#    products, on='product_id', how='left')['department_id_y'].unique()

,product_id,dept_reorder_prob,department_id
0,1,0.574180,0
1,2,0.346721,0
2,3,0.653460,0
3,4,0.541885,0
4,5,0.346721,0


In [34]:
#
#   4: People love to reorder organic 
#
prod_features = prod_features.merge(
    products[['product_id', 'product_name']], on='product_id', how='left')
prod_features['is_organic'] = prod_features.apply(
    lambda x: 1 if 'organic' in x['product_name'].lower() else 0, axis=1)
del prod_features['product_name']
prod_features.head()

,product_id,dept_reorder_prob,department_id,is_organic
0,1,0.574180,0,0
1,2,0.346721,0,0
2,3,0.653460,0,0
3,4,0.541885,0,0
4,5,0.346721,0,0


In [22]:
#
# Generate user features ... 
# FIXME make pretty and organized like prod_features
#

print "Computing user / user-product features ... "

prior_gb_up = prior.groupby(['user_id', 'product_id'])
prior_gb_u = prior.groupby('user_id')
# FIXME is there a way to reuse prior_gb_up here? 

 Computing user / user-product features ... 


In [12]:
#   1: Number of times product has been ordered by user
print "\tnorders of product by user ... ",
user_prod_norders = prior_gb_up.apply(len).reset_index(name='user_prod_norders')
print "done."

	norders of product by user ...  done.


In [13]:
#   2: Total number of orders by user
print "\tnorders by user ... ",
user_norders = prior_gb_u.apply(
    lambda x: len(set(x['order_id']))).reset_index(name='user_norders')
# FIXME you can also get this from orders table ... probably quicker
print "done."

	norders by user ...  done.


In [20]:
#   3: Fraction of user baskets containing product
print "\torder rate of product by user ... ",
user_prod_rate = user_prod_norders.merge(user_norders, on='user_id', how='left')
user_prod_rate['user_prod_rate'] = user_prod_rate['user_prod_norders'] / 1.0 / user_prod_rate['user_norders']

del user_prod_rate['user_prod_norders'], user_prod_rate['user_norders']
# FIXME I guess you could delete the other tables instead, and this is then a bit
# more handy to carry around, but it doesn't play nice with jupyter, where
# I want cell execution independence ...

print "done."

	order rate of product by user ...  done.


In [14]:
#   4: Number of baskets since user last ordered this item
# FIXME why is this so slow?!
print "\tbaskets since last order of product ... ",
baskets_since = prior_gb_up.apply(
    lambda x: max(x['order_number'])).reset_index(name='last_basket')

baskets_since = baskets_since.merge(user_norders, on='user_id', how='left')
baskets_since['baskets_since'] = baskets_since['user_norders'] - baskets_since['last_basket']
del baskets_since['user_norders'], baskets_since['last_basket']
print "done."

	baskets since last order of product ...  done.


In [15]:
#   5: Number of baskets since user first ordered this item
# FIXME why is this so slow?!
print "\tbaskets since first order of product ... ",
baskets_since_1st = prior_gb_up.apply(
    lambda x: min(x['order_number'])).reset_index(name='last_basket')

baskets_since_1st = baskets_since_1st.merge(user_norders, on='user_id', how='left')
baskets_since_1st['baskets_since_1st'] = baskets_since_1st['user_norders'] - baskets_since_1st['last_basket']
del baskets_since_1st['user_norders'], baskets_since_1st['last_basket']
print "done."

	baskets since first order of product ...  done.


In [35]:
#
#   1: Product reorder probability
#

# The way I was originally computing this may have been biased.
# Below is an unbiased (i.e. without peeking at the training
# set) way to estimate the probability a product is ordered in
# a basket given that it ever appeared in a basket.

x = user_prod_norders.merge(
    baskets_since_1st, on=['user_id','product_id'], how='left'
    ).groupby('product_id').apply(
    lambda x: sum( x['user_prod_norders'] ) / 1.0 / (1 + sum( x['baskets_since_1st'] ))
    ).reset_index(name='reorder_prob')

prod_features = prod_features.merge(
    x[['product_id', 'reorder_prob']], on='product_id', how='left')

In [16]:
#   6: Mean basket position. Lower basket position means customer likes product?
print "\tmean basket position ... ",
baskets_pos = prior_gb_up.apply(
    lambda x: x['add_to_cart_order'].mean()).reset_index(name='basket_pos')
print "done."

	mean basket position ...  done.


In [17]:
#   7/8/9: Time of day / dow / days since prior order
print "\torder features ... ",
order_features = orders[orders['eval_set'] != 'prior'][[
    'user_id',
    'order_hour_of_day',
    'order_dow',
    'days_since_prior_order']]

assert len(order_features) == len(orders['user_id'].unique())
print "done."

	order features ...  done.


In [18]:
#    10: Does user reorder in general?
print "\tuser reorder rate ... ",
user_reorder_rate = prior_gb_u.apply(
    lambda x: x['reordered'].sum() / 1.0 / len(x)
    ).reset_index(name='user_reorder_rate')
print "done."

	user reorder rate ...  done.


In [19]:
#   11: User order rate
print "\tuser general order rate (days) ... ",
user_order_dt = prior_gb_u.apply(
    lambda x: x['days_since_prior_order'].mean()
    ).reset_index(name='user_order_dt')
print "done."

	user general order rate (days) ...  done.


In [39]:
#   12: User product order rate.
print "\tuser product order rate (days) ... ",
def up_order_rate(x):
    mn = min(x['order_number'])
    mx = max(x['order_number']) # we actually want max over all products
                                # this will have to do for now
    if mn == mx:
        return 0
    else:
        return sum(x[x['order_number'] >= mn]['days_since_prior_order']) / 1.0 / (mx - mn)

user_prod_order_dt = prior_gb_up.apply(up_order_rate
    ).reset_index(name='user_prod_order_dt')
print "done."

	user product order rate (days) ... 

KeyboardInterrupt: 

In [40]:
#
# Assemble all features together in one table
#
features = user_prod_norders.merge(
    user_norders, on='user_id', how='left'
    ).merge(user_prod_rate, on=['user_id', 'product_id'], how='left'
    ).merge(baskets_since, on=['user_id', 'product_id'], how='left'
    ).merge(baskets_since_1st, on=['user_id', 'product_id'], how='left'
    ).merge(baskets_pos, on=['user_id', 'product_id'], how='left'            
    ).merge(prod_features, on='product_id', how='left'
    ).merge(order_features, on='user_id', how='left'
    ).merge(user_reorder_rate, on='user_id', how='left'
    ).merge(user_order_dt, on='user_id', how='left'
    ).merge(pd.read_csv('order_streaks.csv'), on=['user_id','product_id'], how='left')
# need to keep "product features" for submission stage
    #).merge(user_prod_order_dt, on='user_id', how='left'

# Add training data, i.e., whether product was reordered by user
features = features.merge(
    train[['user_id','product_id','reordered']],
    on=['user_id','product_id'], how='left').fillna(0)

features.head(10)

,user_id,product_id,user_prod_norders,user_norders,user_prod_rate,baskets_since,baskets_since_1st,basket_pos,dept_reorder_prob,department_id,is_organic,reorder_prob,order_hour_of_day,order_dow,days_since_prior_order,user_reorder_rate,user_order_dt,order_streak,reordered
0,1,196,10,10,1.0,0,9,1.400000,0.653460,0,0,0.327370,8,4,14.0,0.694915,20.259259,5.0,1.0
1,1,10258,9,10,0.9,0,8,3.333333,0.574180,0,0,0.295700,8,4,14.0,0.694915,20.259259,5.0,1.0
2,1,10326,1,10,0.1,5,5,5.000000,0.649913,1,1,0.276632,8,4,14.0,0.694915,20.259259,0.0,0.0
3,1,12427,10,10,1.0,0,9,3.300000,0.574180,0,0,0.245536,8,4,14.0,0.694915,20.259259,5.0,0.0
4,1,13032,3,10,0.3,0,8,6.333333,0.560922,0,0,0.190001,8,4,14.0,0.694915,20.259259,1.0,1.0
5,1,13176,2,10,0.2,5,8,6.000000,0.649913,1,1,0.403225,8,4,14.0,0.694915,20.259259,0.0,0.0
6,1,14084,1,10,0.1,9,9,2.000000,0.669969,1,1,0.315376,8,4,14.0,0.694915,20.259259,0.0,0.0
7,1,17122,1,10,0.1,5,5,6.000000,0.649913,1,0,0.220314,8,4,14.0,0.694915,20.259259,0.0,0.0
8,1,25133,8,10,0.8,0,7,4.000000,0.669969,1,1,0.192602,8,4,14.0,0.694915,20.259259,5.0,1.0
9,1,26088,2,10,0.2,8,9,4.500000,0.574180,0,0,0.158341,8,4,14.0,0.694915,20.259259,0.0,1.0


In [42]:
#
# Establish separate training and validation data
#

Nval = 50000 # seems like a good number
uids = train['user_id'].unique()
uids_train = uids[:-Nval]
uids_val = uids[-Nval:]

feature_list = [
    'reorder_prob', # product features
    'dept_reorder_prob',
    'department_id',
    'is_organic',
    'user_prod_norders', # user-product features
    'user_prod_rate',
    'baskets_since',
    'baskets_since_1st',
    'basket_pos',
    'order_dow', # order features
    'order_hour_of_day',
    'days_since_prior_order',
    'user_norders', # user features
    'user_reorder_rate',
    'user_order_dt',
    #'user_prod_order_dt',
    'order_streak']

#
# training
#
where = features['user_id'].isin(uids_train)
X_train = features[where][feature_list].as_matrix()
y_train = features[where]['reordered'].as_matrix()

#
# validation
#
where = features['user_id'].isin(uids_val)
X_val = features[where][feature_list].as_matrix()
y_val = features[where]['reordered'].as_matrix()

#
# test / submission
#
uids_test = orders[orders['eval_set'] == 'test']['user_id'].unique()
where = features['user_id'].isin(uids_test)
X_test = features[where][feature_list].as_matrix()

print len(uids_val), len(uids_train), len(uids_test), len(orders['user_id'].unique())

50000 81209 75000 206209


In [46]:
#
# Compute training target(s)
#
target = train.groupby('user_id'
    ).apply(lambda x: set(x[x['reordered'] == True]['product_id'])
    ).reset_index(name='target')
train_target = target[target['user_id'].isin(uids_train)]
train_features = features[features['user_id'].isin(uids_train)]
val_target = target[target['user_id'].isin(uids_val)]
val_features = features[features['user_id'].isin(uids_val)]

In [ ]:
#
# try a bunch of hyperparameters
#
trees = []
import itertools
for mss, md, mf in itertools.product(
    [1000, 1500], 
    [10, 15, 20], 
    [4, 6]):
    #['gini', 'entropy']):

    clf = RandomForestClassifier(
        criterion='gini',
        n_estimators=15, 
        random_state=0, 
        min_samples_split=mss,
        max_features=mf,
        max_depth=md,
        n_jobs=4,
        verbose=0)
    print clf
    clf.fit(X_train, y_train)
    trees.append(clf)
    #export_graphviz(clf, out_file='tree.dot')
    print clf.feature_importances_
               
    print "Performance on training set ... "
    pred = clf.predict_proba(X_train)[:, 1]
    for p in [0.195, 0.2]: # time and time again, this value wins
        train_features['prediction'] = pred > p
        out = train_features.groupby('user_id').apply(
            lambda x: set(x[x['prediction'] == True]['product_id'])
            ).reset_index(name='prediction')
        out = out.merge(train_target, on='user_id', how='left')
        f1 = f1score(out['prediction'], out['target'])
        print p, f1score(out['prediction'], out['target'])
    
    if f1 < 0.38: continue
    print "Performance on validation set ... "
    pred = clf.predict_proba(X_val)[:, 1]
    for p in [0.195, 0.2]:
        
        val_features['prediction'] = pred > p
        out = val_features.groupby('user_id').apply(
            lambda x: set(x[x['prediction'] == True]['product_id'])
            ).reset_index(name='prediction')
        out = out.merge(val_target, on='user_id', how='left')
        print p, f1score(out['prediction'], out['target'])
        del val_features['prediction']

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=10, max_features=4, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=1000, min_weight_fraction_leaf=0.0,
            n_estimators=15, n_jobs=4, oob_score=False, random_state=0,
            verbose=0, warm_start=False)
[  4.56911117e-02   9.53036935e-03   2.74916309e-03   1.54195495e-04
   1.28045233e-01   2.12984066e-01   1.85575002e-01   2.18027582e-02
   2.98521588e-03   2.19768426e-04   3.97639832e-04   1.63400996e-02
   1.26731185e-02   1.39836534e-02   5.67513534e-03   3.41193470e-01]
Performance on training set ... 


/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


0.195 0.378693715714
0.2 0.378177936485
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=10, max_features=6, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=1000, min_weight_fraction_leaf=0.0,
            n_estimators=15, n_jobs=4, oob_score=False, random_state=0,
            verbose=0, warm_start=False)
[  5.20018506e-02   6.64961308e-03   1.22216548e-03   1.09506985e-04
   7.99708554e-02   3.06104997e-01   1.75367684e-01   2.12090777e-02
   2.50393063e-03   1.39110430e-04   4.29305491e-04   1.91449402e-02
   6.76756133e-03   1.32954339e-02   2.81667584e-03   3.12267292e-01]
Performance on training set ... 
0.195 0.3795489897
0.2 0.379530450826
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=15, max_features=4, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=1000, min

/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


0.195 0.378679031773
0.2 0.378719996591
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=15, max_features=6, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=1000, min_weight_fraction_leaf=0.0,
            n_estimators=15, n_jobs=4, oob_score=False, random_state=0,
            verbose=0, warm_start=False)
[ 0.05676222  0.00779127  0.00105298  0.00042708  0.06887387  0.29650777
  0.16873935  0.02310045  0.00525608  0.00106943  0.00236661  0.02579595
  0.00927752  0.02926409  0.00963624  0.29407908]
Performance on training set ... 
0.195 0.385490504234
0.2 0.385514606946
Performance on validation set ... 
0.195 0.378228002734
0.2 0.378458594657
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=20, max_features=4, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=1000, min_wei

In [ ]:
#
# Let's see how we did
#


In [ ]:
#
# Optional. Retrain the model with the winning hyperparameters on the entire 
# training data set. I think this is always a good idea? Can it go wrong?

#
# This model was somewhat over-fitted on the training data.
# But it had a decent validation score. let's give it more data 
# and hope for the best :/. Nothing else is working ...
#
clf = RandomForestClassifier(
                n_estimators=50, 
                random_state=0, 
                min_samples_split=500,
                max_features=6,
                max_depth=15,
                n_jobs=4,
                verbose=15)
X_all = np.vstack((X_train, X_val))
y_all = np.concatenate((y_train, y_val))
clf.fit(X_all, y_all)

In [ ]:
# check the model hasn't changed too much

print clf.feature_importances_ 
#print trees[2].feature_importances_

uids = np.concatenate((uids_train, uids_val))

where = features['user_id'].isin(uids)
X_all = features[where][feature_list].as_matrix()
pred = clf.predict_proba(X_all)[:, 1]

this_features = features[where]
this_features['prediction'] = pred > 0.195

out = this_features.groupby('user_id').apply(
    lambda x: set(x[x['prediction'] == True]['product_id'])
    ).reset_index(name='prediction')
out = out.merge(target, on='user_id', how='left')

print f1score(out['target'], out['prediction'])
#del this_features['prediction']

In [ ]:
trees[4]

In [ ]:
#
# Let's make a submission!!
#
print clf
#clf = trees[4]

#
# generate predictions for test set from model
#
oid_uid_test = orders[orders['eval_set'] == 'test'][['order_id', 'user_id']]
where = features['user_id'].isin(uids_test)
test_features = features[where]

X_test = test_features[feature_list].as_matrix()
pred = clf.predict_proba(X_test)[:, 1] > 0.195

test_features.loc[:, 'prediction'] = pred
test_features = test_features.merge(oid_uid_test, on='user_id', how='left')
out = this_features.groupby('order_id').apply(
    lambda x: set(x[x['prediction'] == True]['product_id'])
    ).reset_index(name='prediction')

out.head()

In [ ]:
#
# Write predictions to disk for submission.
#

fd = open('submission.csv', 'w')
fd.write('order_id,products\n')

for oid, pr in zip(out['order_id'], out['prediction']):
    fd.write('%d,' % oid)

    if pr:
        fd.write(' '.join(map(str, pr)))
    else:
        fd.write('None')
    fd.write('\n')

fd.close()

In [ ]:
#
#
# POST MORTEM: WHAT WENT WRONG?
#   what does the misclassified data look like?
#   what features could we add to better classify them?
#

#pred = clf.predict_proba(X_train)[:, 1]
#train_features['prediction'] = pred > 0.195
#train_features[train_features['prediction'] != train_features['reordered']].head(25)
def pd_f1score(x):
    # an F1 score routine working with dataframes
    pr = set(x[x['prediction'] == True]['product_id'])
    ob = set(x[x['reordered'] == 1]['product_id'])
    if not pr:
        pr = set([None])

    if not ob:
        ob = set([None])
        
    precision = len(pr & ob) / float(len(pr))
    recall = len(pr & ob) / float(len(ob))

    if precision == recall == 0:
        return 0
    else:
        return 2 * precision * recall / (precision + recall)
    
f1scores = train_features.groupby('user_id').apply(pd_f1score).reset_index(name='f1score')
f1scores.sort_values('f1score').head(25)

In [ ]:


#print prior.groupby('user_id').apply(lambda x: len(set(x['orderf1scores['f1score'].mean()
#tmp = train_features[train_features['user_id'] == 30542].head(25)
tmp = user_reorder_rate.merge(f1scores, on='user_id')
pylab.scatter(tmp['user_reorder_rate'], tmp['f1score'], s=5)
pylab.show()
#tmp[tmp['prediction'] != tmp['reordered']].head(25)

In [ ]:
errors = train_features.groupby('product_id').apply(
    lambda x: pd.Series(
        {'false_pos':sum((x['prediction'] == 1) & (x['reordered'] == 0)),
         'false_neg':sum((x['prediction'] == 0) & (x['reordered'] == 1)),
         'true_pos':sum((x['prediction'] == 1) & (x['reordered'] == 1)),
         'true_neg':sum((x['prediction'] == 0) & (x['reordered'] == 0))})
).reset_index().merge(pd.read_csv('products.csv'), on='product_id')

In [ ]:
errors['recall'] = errors['true_pos'] / 1.0 / (errors['false_neg'] + errors['true_pos'])
errors['precision'] = errors['true_pos'] / 1.0 / (errors['false_pos'] + errors['true_pos'])
errors['f1score'] = 2 * errors['precision'] * errors['recall'] / (errors['precision'] + errors['recall'])

errors[errors['false_pos'] + errors['true_pos'] > 50].sort_values('f1score')


In [ ]:
tmp = prior.groupby('product_id').apply(len).reset_index(name='norders')
tmp = tmp.merge(errors, on='product_id', how='inner')
tmp.sort_values('norders', ascending=0)

In [ ]:
#t = prior.groupby('product_id').apply(len)
pylab.hist(t, bins=np.linspace(1, 100, 100))
pylab.show()

In [ ]:
# the product f1 score is not quite the same as the user f1score
print errors['f1score'].mean()

# I don't know if one tracks the other ... hopefully

# Let's retrain part of the classifier on products we performed poorly
# on. Kind of like making a hyper-tree where this is the first split
bad_products = errors[errors['f1score'] < 0.4]['product_id']
print len(bad_products)

# Are there a lot of these? Enough to think it will matter?
print len( features[features['product_id'].isin(bad_products)] )
print len(features)

In [ ]:
# Although these are only 1/10th of the product selection, they amount to
# almsot 1/3rd of the orders ...
retrain_features = train_features[train_features['product_id'].isin(bad_products)]
X_retrain = retrain_features[feature_list].as_matrix()
y_retrain = retrain_features['reordered'].as_matrix()
old_clf = clf

In [ ]:
retrees = []
for mss in [500, 1000, 1500]:
    for md in [6, 10, 15]:
        for mf in [4, 6]:
            print mss, md, mf
            reclf = RandomForestClassifier(
                n_estimators=15, 
                random_state=0, 
                min_samples_split=mss,
                max_features=mf,
                max_depth=md,
                n_jobs=4,
                verbose=0)
            reclf.fit(X_retrain, y_retrain)
            retrees.append(reclf)
            #export_graphviz(clf, out_file='tree.dot')
            print reclf.feature_importances_

In [ ]:
# compute training target(s)

# first, deep tree
pred = clf.predict_proba(X_train)[:, 1]
train_features['prediction'] = pred > 0.195

# first, deep tree
pred = clf.predict_proba(X_val)[:, 1]
val_features['prediction'] = pred > 0.195

In [ ]:
where_t = train_features['product_id'].isin(bad_products)
X_retrain = train_features[where_t][feature_list].as_matrix()
where_v = val_features['product_id'].isin(bad_products)
X_reval = val_features[where_v][feature_list].as_matrix()

print "Original model:",
f1scores = train_features.groupby('user_id').apply(
    pd_f1score).reset_index(name='f1score')
print f1scores['f1score'].mean(), "(training)"
f1scores = val_features.groupby('user_id').apply(
    pd_f1score).reset_index(name='f1score')
print f1scores['f1score'].mean(), "(validation)"
print

print "Boosted model:"

for _clf in retrees:
    print _clf

    _pred = _clf.predict_proba(X_retrain)[:, 1]
    for p in [0.195]: # time and time again, this value wins
        
        train_features.loc[where_t, 'prediction'] = _pred > p
        f1scores = train_features.groupby('user_id').apply(
            pd_f1score).reset_index(name='f1score')
        print f1scores['f1score'].mean(), "(training)", "(training)"        
        
        #f1scores = train_features.groupby('user_id').apply(
        #    pd_f1score).reset_index(name='f1score')
        #print p, f1scores['f1score'].mean()
    
    print "Performance on validation set ... "
    _pred = _clf.predict_proba(X_reval)[:, 1]
    for p in [0.195]:
        
        val_features.loc[where_v, 'prediction'] = _pred > p
        f1scores = val_features.groupby('user_id').apply(
            pd_f1score).reset_index(name='f1score')
        print p, f1scores['f1score'].mean()

In [ ]:
#
# Let's make a submission!!
#
print clf
_clf = trees[1]
print _clf

#
# generate predictions for test set from model
#
oid_uid_test = orders[orders['eval_set'] == 'test'][['order_id', 'user_id']]
where = features['user_id'].isin(uids_test)
test_features = features[where]
where_p = test_features['product_id'].isin(bad_products)

X_test = test_features[feature_list].as_matrix()
pred = clf.predict_proba(X_test)[:, 1] > 0.195

test_features.loc[:, 'prediction'] = pred

X_retest = test_features[where_p][feature_list].as_matrix()
_pred = _clf.predict_proba(X_retest)[:, 1] > 0.195
test_features.loc[where_p, 'prediction'] = _pred

test_features = test_features.merge(oid_uid_test, on='user_id', how='left')

In [ ]:
out = test_features.groupby('order_id').apply(
    lambda x: set(x[x['prediction'] == True]['product_id'])
    ).reset_index(name='prediction')

out.head()

In [ ]:
#
# Write predictions to disk for submission.
#

fd = open('submission.csv', 'w')
fd.write('order_id,products\n')

for oid, pr in zip(out['order_id'], out['prediction']):
    fd.write('%d,' % oid)

    if pr:
        fd.write(' '.join(map(str, pr)))
    else:
        fd.write('None')
    fd.write('\n')

fd.close()